# Build a language model for a specific subject


One of the state-of-the art solutions to build NLP application is using word embeddings to
compute similarities between texts. Generally, they are vector representations of words that
are capable of capturing the context of a word in a document or relation with other words.

Word2vec is a two-layer neural network that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. The purpose and usefulness of Word2vec is to group the vectors of similar words together in vectorspace. This way. it is able to detect similarities mathematically [1].

A pre-trained set of vectors containing part of Google News dataset can be downloaded and used [2].
In case of technical fields, however, in order to achieve a better accuracy, a new language model should be trained,
in order to obtain the specific vocabulary that may be missing from the news dataset.

## Build up an input corpus

The first step is to build a corpus of specialized words, that can be used to get the vocabulary.
In order to do that, we decided to use papers stored in Zenodo [3], since it offers a free end point for
querying. In order to do that, we raise multiple queries to the server and we store the results locally.
The approach that we selected was to download the pdf files and transform them to txt, using a command lines tool
for Linux. To speed up the process and to avoid duplicates, we save the files in the '/tmp' folder and check, each time, if the file is already there. In case it is, we load the text, which is already pre-processed. I case of builing a language model, it is better to not have duplicates, since it can change the values attached to words.

The next step is to cleanup the text. In order to to that, the first step is to remove all punctation and replace non-alpha characters to spaces. Them all the words are transformed to lowercase and lemmatized to their basis word. Then,
all the stop words are removed, together with the non-English and short words. In this case, it is important to keep the  abbreviations, technical words which are not the English dictionary and short words related to the concerned field.
Them all the abreviations are replaced by the corresponding words sequence, to be able to compute similarities right. The clean-up texts are then saved in the corresponding text files in the '/tmp' folder.

## Build a language model

The corpus of documents is directly sent to word2vec library of gensim [4] and then saved in a binary file for further
use.


[1] https://skymind.ai/wiki/word2vec
[2] https://code.google.com/archive/p/word2vec/
[3] https://zenodo.org/
[4] https://radimrehurek.com/gensim/models/word2vec.html

In [5]:
%run "NLP_clustering.ipynb"
%run "Utils_Zenodo.ipynb"

def get_entries_from_zenodo(query):
    
    solar_observations_list_of_words = ["solar irradiance", "solar", "photovoltaic", ]
    
    total_articles = get_zenodo_entries(query)
    documents = []
    
    for article in total_articles:
        doi = article['doi']
        mod_doi = doi.replace('/', '-')

        #look in tmp folder if there is a file containing the doi
        # if there is, just read the file and move to the next entry
        if os.path.isfile('/tmp/' + doi + ".txt"):
            with open('/tmp/' + doi + ".txt") as f:
                doc_list = f.read().split(' ')
        else:
            print(article['files'][0]['links']['self'])
            # download the file and preprocess in the information
            doc_list = save_pdf_and_get_text(article['files'][0]['links']['self'])
            # overwrite the doi temporary file
            with open('/tmp/' + mod_doi + ".txt", 'w') as f:
                f.write(' '.join(doc_list))
                f.close()
            # add the text to the corpus   
            if doc_list:
                documents.append(doc_list)
 
    return documents


if __name__ == "__main__":
    #documents is a list of lists
    documents = get_entries_from_zenodo(SEARCH_QUERY)
    documents.extend(get_entries_from_zenodo("solar"))
    documents.extend(get_entries_from_zenodo("photovoltaic"))
    documents.extend(get_entries_from_zenodo("Renewable Energy"))
    documents.extend(get_entries_from_zenodo("solar pond"))
    documents.extend(get_entries_from_zenodo("solar observations"))
    documents.extend(get_entries_from_zenodo("global horizontal irradiance"))
    documents.extend(get_entries_from_zenodo("irradiance"))
    documents.extend(get_entries_from_zenodo("solar collector"))
    documents.extend(get_entries_from_zenodo("downwelling longwave"))
    documents.extend(get_entries_from_zenodo("clear sky"))
    documents.extend(get_entries_from_zenodo("diffuse radiation"))
    documents.extend(get_entries_from_zenodo("water vapor"))
    documents.extend(get_entries_from_zenodo("snowdepth"))
    documents.extend(get_entries_from_zenodo("bi-directional reflectance"))
    documents.extend(get_entries_from_zenodo("snowcover"))
    documents.extend(get_entries_from_zenodo("airmass"))
    documents.extend(get_entries_from_zenodo("downwelling"))
    
    model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
    model.train(documents,total_examples=len(documents),epochs=10)
    model.wv.save_word2vec_format(MODEL_TO_TRAIN, binary=True)
    

/home/researchyou2/iulia/Geo_Classifier
/home/researchyou2/iulia/Geo_Classifier
https://zenodo.org/api/files/e64f828f-85f5-4bd2-b33b-50e6d5be39ca/coatings-09-00593-v3.pdf
https://zenodo.org/api/files/c4c448cb-bdd1-455c-9ddc-f9a010b7e5f0/IEEEJPV_Wietler_181019_final_DISC.pdf
https://zenodo.org/api/files/a5d611d5-5322-4f3f-b2bc-665d1bf66fbd/2018_2_SE%20-%20Cost-Benefit%20Reconfigurable%20Modules%20-%20preprint%20%28black%29.pdf
https://zenodo.org/api/files/104eb429-f512-4ac4-88ec-48fb29f46ad9/SE2017_ThermochemicalRedoxCycles.pdf
https://zenodo.org/api/files/9f98f128-e365-4f29-a20c-460e81227c60/1-s2.0-S0038092X19309065-main.pdf
https://zenodo.org/api/files/0944beda-2a9f-446b-8d1a-042edc6692f0/Solar%20Energy-accepted%20with%20minor.pdf
https://zenodo.org/api/files/afb493ce-3f47-4c45-b9d8-71175ace618e/OP_simu_rearreflection.pdf
https://zenodo.org/api/files/e733d152-40a6-4a48-84b3-bccb5cdf5755/SE2017_HighTemperatureH2Production.pdf
https://zenodo.org/api/files/6cda562d-4d8f-4d93-8f77-d01ef5a

https://zenodo.org/api/files/c31b880b-671e-4836-aa58-e9de0aba7ce8/7761.pdf
https://zenodo.org/api/files/69c6c672-c959-447a-bfd9-59da257c2614/2890.pdf
https://zenodo.org/api/files/78e93ec2-be44-41f1-bd83-6d72faef4157/Solar%20energy%20a%20promising%20source%20for%20%20%20E-vehicles1.pdf
https://zenodo.org/api/files/b8ddb252-cefa-4dd9-9c80-91ccda53e9a5/67%20The%20Application%20of%20Solar%20Energy%20in%20Agricultural%20Systems.pdf
https://zenodo.org/api/files/930866c6-e9d7-47c9-8432-c7d35b4d6ffb/article.pdf
https://zenodo.org/api/files/bc01d593-4731-4b8b-b3cf-2498fc241efa/22_IJRG18_A08_1622.pdf
https://zenodo.org/api/files/33a5f2f8-b41b-451a-a810-aff5e969075e/10004281.pdf
https://zenodo.org/api/files/549694b6-2098-4eb7-a4ef-1bd64bc092b6/20150077-pv%20module%20performance.PDF
https://zenodo.org/api/files/32ea821c-66bb-44f1-9c3d-20958b5495b1/9998009.pdf
https://zenodo.org/api/files/2e2ffb13-f4cb-4e91-b728-8bb544651c37/updated%2812-17%29paperid-format.pdf
https://zenodo.org/api/files/dd71e39a

https://zenodo.org/api/files/4cd691cd-c700-4de5-96ed-7949f7ba8998/1-s2.0-S0927024818305336-main.pdf
https://zenodo.org/api/files/9ca4a5a7-3bf6-40a7-85e1-9fa22fdfc959/final_version.pdf
https://zenodo.org/api/files/baeeab43-5b43-4af3-add8-bde426b556fd/2018%20manuscript_method_Sb2Se3_preprint.pdf
https://zenodo.org/api/files/53555f38-b912-4bfc-be49-e0e44c233b3e/Full%20version%20-%20Ge%20based%20kesterite%20CZGSe.pdf
https://zenodo.org/api/files/dfc8fee2-ded0-4902-8a20-d6da903a4aa8/paper%20on%20soiling.pdf
https://zenodo.org/api/files/0690247b-2e54-43a7-8e31-686a2be9dfad/10001136.pdf
https://zenodo.org/api/files/ea41f206-0198-4b8f-8a79-eacbb52d4e6c/10003627.pdf
https://zenodo.org/api/files/96fd04bb-9303-473c-b62c-71697b588c6d/1746.pdf
https://zenodo.org/api/files/06463f02-7aab-4c31-aa29-45016a342c94/180%20A%20Review%20on%20Dye%20Sensitized%20Solar%20Cell.pdf
https://zenodo.org/api/files/a3ec06a7-9ee8-4e3e-bcde-e5ff9021938d/17350.pdf
https://zenodo.org/api/files/4718603d-e432-428c-9fe9-93bd

https://zenodo.org/api/files/81dabb27-8039-41ee-9ea0-149daacde18d/Paper12.pdf
https://zenodo.org/api/files/f47e8118-ebe7-4468-b3bc-dda368f261d4/9999225.pdf
https://zenodo.org/api/files/80cc74a9-b1a9-4b47-ba65-9c4744b90126/12732.pdf
https://zenodo.org/api/files/f7df39f2-77c2-46dc-b2a9-e0be1bbd45b5/7EO.2.5.pdf
https://zenodo.org/api/files/97509297-0571-445f-a273-59bc60241f38/6DO.7.1.pdf
https://zenodo.org/api/files/b1aa19f9-575d-4b3a-8f4a-d52a3528d048/redin.pdf
https://zenodo.org/api/files/1753b22d-4a0e-4fdf-94d2-44f6f403b091/12276.pdf
https://zenodo.org/api/files/07b29497-7036-48cd-811b-794e5602c5cf/10006771.pdf
https://zenodo.org/api/files/0c253db1-34d3-4487-94ca-4ba5f5f3c1ef/8064.pdf
https://zenodo.org/api/files/d48ac8b2-5582-49b1-b79e-d1d2a330f657/%2816-32%29Paper11_1_Revised%201-format.pdf
https://zenodo.org/api/files/d395a697-dae6-4262-a96f-c35807fce7b7/62.PDF
https://zenodo.org/api/files/89884495-4a05-4313-8618-62c3baf09587/10004926.pdf
https://zenodo.org/api/files/248b6c35-4692-4

https://zenodo.org/api/files/39279c05-e1b7-4fc9-937f-8822b86bc589/2016_07_Philipps_CPVMatch_CPV12_Postprint.pdf
https://zenodo.org/api/files/b2f59594-a468-4aa7-9d6c-2be2ce14aa1a/10004260.pdf
https://zenodo.org/api/files/b6490d03-f244-4345-9968-08b84c26628c/9998743.pdf
https://zenodo.org/api/files/9a21569d-11ad-427d-b6cc-49b1f9d075e1/5024.pdf
https://zenodo.org/api/files/fc07bc08-41fc-4198-8cb1-6f3f3d8b7158/10000605.pdf
https://zenodo.org/api/files/12bc8e54-0d81-4a86-80e7-7d0262d595b8/10000605.pdf
https://zenodo.org/api/files/30e8e54e-27c4-4ba4-949c-5cba8352b8de/10004240.pdf
https://zenodo.org/api/files/ae31f06b-9f15-4e28-a33c-e4eb409aca09/95.pdf
https://zenodo.org/api/files/95b04e64-61b8-41d7-b25b-6e3058c6b86c/%2829-40%29A%20Review%20on%20Augmentation-format%20%281%29.pdf
https://zenodo.org/api/files/a8adcaa2-76e8-404a-baaf-42083de24a63/4091.pdf
https://zenodo.org/api/files/82dcf01f-354a-4aa6-9b3f-f2c2767ef601/Theoretical%20and%20experimental%20analysis%20of%20solar%20enclosure%20as%20

...

https://zenodo.org/api/files/3be16ab9-11c5-4e42-8562-041b1786a58c/5967.pdf
https://zenodo.org/api/files/a79c92f4-d924-4dd9-9cd3-9e38d43d8d24/Vol%207%281%29%20-%20Cont.%20J.%20Appl%20Sci%207-13.pdf
https://zenodo.org/api/files/e661f2dd-3ff6-4b91-a381-de14bb9ad340/9997866.pdf
https://zenodo.org/api/files/6ae6b590-2d4e-4f87-873f-ed3e2d15bc5b/7309.pdf
https://zenodo.org/api/files/905949d4-efbb-4593-bbe5-d9abac161605/9064.pdf
https://zenodo.org/api/files/0dfe53bd-b590-45b0-a5cc-37f4d0ca4b3b/10010069.pdf
https://zenodo.org/api/files/27d7e247-57e4-4db5-ab45-535955d452b3/10002741.pdf
https://zenodo.org/api/files/49ec0ac8-d97d-47d8-8fa7-e47c49985a98/CS19_llama_proceedings.pdf
https://zenodo.org/api/files/729485ea-b830-4749-91de-69ad4555537b/10.21307_ijssis-2017-503.pdf
https://zenodo.org/api/files/22ed6317-a1bd-49d3-9f23-90cc36ee596a/Modeling%20of%20Multi%20Junction%20Solar%20Cell%20and%20MPPT%20Methods.pdf
https://zenodo.org/api/files/21a41553-a340-450c-ab8c-7fa435e91a58/Speaker08_Llama_CS19.p

https://zenodo.org/api/files/1ea49e2d-7473-4352-88a2-3f9cb946667d/Edited%20Manuscript.pdf
https://zenodo.org/api/files/90aa8f29-321e-40a7-9578-8f22884854fe/Plakat_nr_1_Konferencja_Krynica-Zdr%C3%B3j_2019.pdf
https://zenodo.org/api/files/e762dfcf-92e1-4379-83fc-3d6ee4191790/Study%20into%20predicted%20efficiency%20of%20the%20application%20of%20hybrid%20solar%20collectors%20to%20supply%20energy%20to%20multi-apartment%20buildings.pdf
https://zenodo.org/api/files/2a976f3f-7638-4605-bcdb-e8358cf9d1e2/IRJAES-V4N2P677Y19.pdf
https://zenodo.org/api/files/15cb99be-11dc-43e3-b6ac-5dc2fd968a61/10003767.pdf
https://zenodo.org/api/files/95f9953d-4245-4297-9915-577bd7967f27/10005380.pdf
https://zenodo.org/api/files/e69e8e92-3c4e-42d5-ba21-07aefa8d2012/7098.pdf
https://zenodo.org/api/files/69a51616-4e6a-4973-839f-076c0c45eca6/10010613.pdf
https://zenodo.org/api/files/7acba36a-b805-4f62-8927-19fbacbd136c/10004361.pdf
https://zenodo.org/api/files/f2677fbc-c47e-45c4-ac8f-c631d85b7a1c/10005380.pdf
https:/

https://zenodo.org/api/files/c1691e2a-f051-4984-a2d2-9b425ebd19d0/A.B.S.%202018_Bern-Proceedings-A.C.article.pdf
https://zenodo.org/api/files/0cb040de-1fe7-4783-be5a-429a33916138/10001618.pdf
https://zenodo.org/api/files/8f24c919-e27e-477b-b121-2af65b7b03c5/article.pdf
https://zenodo.org/api/files/bcac9d35-b1ca-43d7-bace-ed7bc453e0b7/article.pdf
https://zenodo.org/api/files/e9ded9d3-22b4-4b19-92b5-d9ea9567c365/T4.2_Mlawer_HITRAN_2008.pdf
https://zenodo.org/api/files/2e282098-55ea-40fc-9b76-09035dec915e/6-DeRidder-OA.pdf
https://zenodo.org/api/files/807b15c1-6493-4b90-b585-893f07986538/Catalysis%20Today%20287%20%282017%29%2015%E2%80%9321.pdf
https://zenodo.org/api/files/3e41b528-7dd8-43e0-b365-cdfe22698b39/APPLICATE_D2.3.pdf
https://zenodo.org/api/files/69c6c672-c959-447a-bfd9-59da257c2614/2890.pdf
https://zenodo.org/api/files/f0b977d3-cd3c-4134-9204-6461c45aabc7/4623.pdf
https://zenodo.org/api/files/7df90e7b-f24b-47ce-af0f-cb1f04310464/10008953.pdf
https://zenodo.org/api/files/3db4559e

https://zenodo.org/api/files/246078b5-cf1c-4be2-a2ba-61a1f9553bd0/14999.pdf
https://zenodo.org/api/files/1e3c36ce-9645-44ad-b4ce-b4b5a64a62c8/10005349.pdf
https://zenodo.org/api/files/503f6384-7c51-474d-8aa6-60fb52af1324/article.pdf
https://zenodo.org/api/files/58ede54d-b959-43be-9403-07bcee8fca5e/10007612.pdf
https://zenodo.org/api/files/8747df62-9630-4f0b-a0a8-6d24b4570d28/T2.7-Fomin_HITRAN_2010.pdf
https://zenodo.org/api/files/3e5ac129-3c70-4ca0-a939-b135960b7413/9254.pdf
https://zenodo.org/api/files/2a5f4686-3b0a-4124-a7be-6a7e2a48403e/5877.pdf
https://zenodo.org/api/files/b74d0dc1-0f23-49ac-8006-bcd3cbc47e8e/10.5755_j01.ms.22.3.7433.pdf
https://zenodo.org/api/files/b4cba6dc-b764-4fd8-b463-257d98011c4c/article.pdf
https://zenodo.org/api/files/ae81fa0d-6e30-4dd8-87e5-327f8e1aa110/9998925.pdf
https://zenodo.org/api/files/a8bf3dac-0822-4151-9832-6431496671ae/16596.pdf
https://zenodo.org/api/files/e27bade2-24d8-474c-829c-9f55c18e7b6a/Hierarchically%20porous%203D-printed%20akermanite%20

https://zenodo.org/api/files/c7172f30-b95a-45f5-9848-407a31883612/tc-13-2001-2019.pdf
https://zenodo.org/api/files/6101648a-d283-4163-b95f-dd949a73dc8a/article.pdf
https://zenodo.org/api/files/e9ded9d3-22b4-4b19-92b5-d9ea9567c365/T4.2_Mlawer_HITRAN_2008.pdf
https://zenodo.org/api/files/b8b4210e-c1d3-4052-8845-af07067d1b79/Mosquera%20Gim%C3%A9nez%20%C3%81%20_%20Ocean%20Circulation%20over%20Formigas%20and%20Ormonde%20Seamounts.pdf
https://zenodo.org/api/files/7d197af0-9f29-4527-8085-fcf2d74e0fa2/Granata.pdf
https://zenodo.org/api/files/e1c4cdd8-0b47-48b9-a572-efcb43a6cbd5/Clippele2018_Article_TheEffectOfLocalHydrodynamicsO.pdf


In [6]:
print(len(model.wv.vocab))
#print(documents)

#print(model.wv.most_similar('solar'))
#print(model.wv.most_similar('photovoltaic'))

print(model.wv.most_similar('irradiance'))
print(model.wv.most_similar('solar'))
print(model.wv.most_similar('photovoltaic'))

model.wv.most_similar("sky")



14070
[('monthly', 0.4758530557155609), ('radiance', 0.4315311908721924), ('radiation', 0.42659685015678406), ('sky', 0.4157203435897827), ('equivalent', 0.41472652554512024), ('incidence', 0.4088061451911926), ('hourly', 0.40874576568603516), ('diffuse', 0.40190279483795166), ('irradiation', 0.3912084400653839), ('rely', 0.38333001732826233)]
[('whim', 0.3435713052749634), ('photovoltaic', 0.3403465151786804), ('areal', 0.31585296988487244), ('hybrid', 0.29691949486732483), ('renewable', 0.2937639355659485), ('navel', 0.2888389229774475), ('gratify', 0.28259822726249695), ('thermoelectric', 0.274077445268631), ('innovative', 0.27404722571372986), ('daylighting', 0.27017486095428467)]
[('solar', 0.3403465151786804), ('integrate', 0.3355311155319214), ('thermoelectric', 0.3348540961742401), ('aggressively', 0.33153194189071655), ('electronics', 0.31924301385879517), ('implementable', 0.31755873560905457), ('emulator', 0.3134787380695343), ('developed', 0.3123370409011841), ('multilevel'

[('cloud', 0.6123994588851929),
 ('overcast', 0.6105839014053345),
 ('cloudless', 0.5887928009033203),
 ('incidence', 0.5727969408035278),
 ('equator', 0.553513765335083),
 ('sunny', 0.5533175468444824),
 ('diffuse', 0.550754189491272),
 ('cloudy', 0.5415629744529724),
 ('hemispherical', 0.5315818190574646),
 ('daylight', 0.5210805535316467)]

In [7]:
print(len(model.wv.vocab))
print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

